In [1]:
import torch

In [7]:
import sys
import numpy as np

In [3]:
sys.path.insert(0, 'examples/pytorch/vit/ViT-quantization/ViT-pytorch')

In [4]:
from models.modeling import VisionTransformer, CONFIGS

In [6]:
config = CONFIGS['ViT-B_16']
model = VisionTransformer(config, 384, zero_head=False, num_classes=1000)

transformer.embeddings.position_embeddings
transformer.embeddings.cls_token
transformer.embeddings.patch_embeddings.weight
transformer.embeddings.patch_embeddings.bias
transformer.encoder.layer.0.attention_norm.weight
transformer.encoder.layer.0.attention_norm.bias
transformer.encoder.layer.0.ffn_norm.weight
transformer.encoder.layer.0.ffn_norm.bias
transformer.encoder.layer.0.ffn.fc1.weight
transformer.encoder.layer.0.ffn.fc1.bias
transformer.encoder.layer.0.ffn.fc2.weight
transformer.encoder.layer.0.ffn.fc2.bias
transformer.encoder.layer.0.attn.query.weight
transformer.encoder.layer.0.attn.query.bias
transformer.encoder.layer.0.attn.key.weight
transformer.encoder.layer.0.attn.key.bias
transformer.encoder.layer.0.attn.value.weight
transformer.encoder.layer.0.attn.value.bias
transformer.encoder.layer.0.attn.out.weight
transformer.encoder.layer.0.attn.out.bias
transformer.encoder.layer.1.attention_norm.weight
transformer.encoder.layer.1.attention_norm.bias
transformer.encoder.layer.1.ff

In [ ]:
model.load_from(np.load(args.pretrained_dir))

In [8]:
model

VisionTransformer(
  (transformer): Transformer(
    (embeddings): Embeddings(
      (patch_embeddings): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): Encoder(
      (layer): ModuleList(
        (0): Block(
          (attention_norm): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (ffn_norm): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (ffn): Mlp(
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (attn): Attention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (out): Linear(in_features=768, out_features=768, bias=True)
     

In [10]:
pretrained_npz = np.load('ViT-B_16.npz')

['Transformer/encoder_norm/bias',
 'Transformer/encoder_norm/scale',
 'Transformer/encoderblock_0/LayerNorm_0/bias',
 'Transformer/encoderblock_0/LayerNorm_0/scale',
 'Transformer/encoderblock_0/LayerNorm_2/bias',
 'Transformer/encoderblock_0/LayerNorm_2/scale',
 'Transformer/encoderblock_0/MlpBlock_3/Dense_0/bias',
 'Transformer/encoderblock_0/MlpBlock_3/Dense_0/kernel',
 'Transformer/encoderblock_0/MlpBlock_3/Dense_1/bias',
 'Transformer/encoderblock_0/MlpBlock_3/Dense_1/kernel',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/key/bias',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/key/kernel',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/out/bias',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/out/kernel',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/query/bias',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/query/kernel',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/value/bias',
 'Tr

In [52]:
l1, l2 = [k for k in model.state_dict()], list(pretrained_npz.keys())

l1.sort()
l2.sort()

i = 0

new_l2 = []
for k1, k2 in zip(l1, l2):
    k2 = k2.replace('Transformer', 'transformer')

    items = k2.split('/')
    items = [item.replace('_', '.') for item in items]
    items = ('.'.join(items)).split('.')

    for i, item in enumerate(items):
        items[i] = item.replace('MultiHeadDotProductAttention', 'attn')
        items[i] = items[i].replace('kernel', 'weight')
        items[i] = items[i].replace('scale', 'weight')
        items[i] = items[i].replace('encoderblock', 'encoder.layer')
        
    k2 = '.'.join(items)
    new_l2.append(k2)
    i += 1


l2 = new_l2
new_l2 = []
for i, k in enumerate(l2):
    if k == 'cls':
        l2[i] = 'transformer.embeddings.cls_token'
    elif k == 'embedding.bias':
        l2[i] = 'transformer.embeddings.patch_embeddings.bias'
    elif k == 'embedding.weight':
        l2[i] = 'transformer.embeddings.patch_embeddings.weight'
    elif k == 'transformer.encoder.norm.bias':
        l2[i] = 'transformer.encoder.encoder_norm.bias'
    elif k == 'transformer.encoder.norm.weight':
        l2[i] = 'transformer.encoder.encoder_norm.weight'
    elif k == 'transformer.posembed.input.pos.embedding':
        l2[i] = 'transformer.embeddings.position_embeddings' 
    else:

        l2[i] = l2[i].replace('LayerNorm.0', 'attention_norm')
        l2[i] = l2[i].replace('LayerNorm.1', 'attention_norm')
        l2[i] = l2[i].replace('LayerNorm.2', 'attention_norm')
        # print(l2[i])
    # new_l2.append(l2[i])

# l2 = new_l2
l1.sort()
l2.sort()

for k1, k2 in zip(l1, l2):
    print(k1, k2)

head.bias head.bias
head.weight head.weight
transformer.embeddings.cls_token transformer.embeddings.cls_token
transformer.embeddings.patch_embeddings.bias transformer.embeddings.patch_embeddings.bias
transformer.embeddings.patch_embeddings.weight transformer.embeddings.patch_embeddings.weight
transformer.embeddings.position_embeddings transformer.embeddings.position_embeddings
transformer.encoder.encoder_norm.bias transformer.encoder.encoder_norm.bias
transformer.encoder.encoder_norm.weight transformer.encoder.encoder_norm.weight
transformer.encoder.layer.0.attention_norm.bias transformer.encoder.layer.0.MlpBlock.3.Dense.0.bias
transformer.encoder.layer.0.attention_norm.weight transformer.encoder.layer.0.MlpBlock.3.Dense.0.weight
transformer.encoder.layer.0.attn.key.bias transformer.encoder.layer.0.MlpBlock.3.Dense.1.bias
transformer.encoder.layer.0.attn.key.weight transformer.encoder.layer.0.MlpBlock.3.Dense.1.weight
transformer.encoder.layer.0.attn.out.bias transformer.encoder.layer.